# Methods

In [167]:
def get_entries(soup):
    #     entries = soup.find_all(name='div', attrs={'class':'row'}) # This way we can have a certain criteria
    entries = soup.find_all(name='div', class_='row') # This looks cleaner
    return entries
    
def get_job_title(entry):
    job_title_container = entry.find(name='a', attrs={'data-tn-element':'jobTitle'})
    job_title = job_title_container.text
    return job_title.strip()

def get_company(entry):
    company_list = []
    try:
        test_entry = entry.find(class_='company')
        company_list.append(test_entry.text.strip()) 
        company = company_list.pop()
    except:
        try:
            test_entry = entry.find(class_='result-link-source')
            company_list.append(test_entry.text.strip()) 
            company = company_list.pop()
        except:
            company = ' '
    return company

def get_location_info(entry):
    company_info = entry.find(class_='sjcl')
    location_info = company_info.find(class_='location')
    
    location = location_info.text.strip()

    # extract neightborhood info if it's there
    neighborhood = get_neighborhood(location_info)
    location = location.rstrip(neighborhood)
    neighborhood = neighborhood.strip('()')

    # extract the zipcode from location if it's there
    zipcode = get_zipcode(location)
    location = location.strip(zipcode)
    
    city, state = get_city_and_state(location)
    
    return city, state, zipcode, neighborhood

def get_city_and_state(location):
    city_state = location.split(', ')
    state = city_state.pop()
    city = city_state.pop()
    return city, state

def get_neighborhood(location_info):
    neighborhood_info = location_info.find(name='span')
    neighborhood = ' '
    if neighborhood_info:
        neighborhood = neighborhood_info.text
    return neighborhood

def get_zipcode(location):
    zipcode = ' '
    temp = [ s for s in location.split() if s.isdigit() ]
    if temp:
        zipcode = temp.pop()
    return zipcode
    
def get_salary(entry):
    salary_list = []
    salary = ''
    try:
        salary_list.append(entry.find('nobr').text.strip())
        salary = salary_list.pop()
    except:
        try:
            salary_container = entry.find(name='div', class_='salarySnippet')
            salary_temp = salary_container.find(name='span', class_='salary')
            salary_list.append(salary_temp.text.strip())
            salary = salary_list.pop()
        except:
            salary = ' '
    return salary

def get_job_summary(entry):
    return entry.find(class_='summary').text.strip()

def get_link(entry):
    link = entry['data-jk']
    return link 

def get_job_description(job_page):
    page = requests.get(job_page)
    time.sleep(1)  #ensuring at least 1 second between page grabs
    soup = BeautifulSoup(page.text, 'lxml')

    # Loop over posts/entries
    description = soup.find(name='div', id='jobDescriptionText')

    description = description.replace('\n',' ')
    description = description.replace('\t',' ')
    return description


In [63]:
# This works as of 08/24/2019
# Used https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b as motivation
# Had to change a few things as indeed structure has changed a bit since

import time
import pandas as pd
import requests
# import bs4
from bs4 import BeautifulSoup


In [2]:
import pandas as pd
test = pd.read_csv('test.csv')

In [3]:
test

,Unnamed: 0,city,job_title,company_name,location,sub_location,description,salary,link
0,1,Boston,Data Scientist,BD,"Boston, MA",NaN,Job Description Summary Digital Health is a bu...,NaN,4631c716fc96075a
1,2,Boston,Data Scientist Analyst,Lincoln Financial,"Boston, MA 02109",Central area,"Alternate Locations: Boston, MA (Massachusetts...",NaN,0873f4263546b21a
2,3,Boston,Associate Consultant,ZoomRx Inc.,"Boston, MA",NaN,Associate Consultant (Business Analysis Team)B...,NaN,3036257274d70543
3,4,Boston,Translational Research Scientist (with proteom...,Novartis,"Cambridge, MA",NaN,#1! That’s the ranking that Fast Company gave ...,NaN,9654c78756072004
4,5,Boston,"Scientist, Neurodegenerative Diseases",Novartis,"Cambridge, MA",NaN,50 million people worldwide are directly affec...,NaN,f88d5149cb65de49
5,6,Boston,Computational Biologist/Data Scientist,Goldfinch Bio,"Cambridge, MA",NaN,Goldfinch Bio is a biotechnology company that ...,NaN,f14bc6dec8b4f60f
6,7,Boston,Data Scientist (Full-Time),proton.ai,"Boston, MA",NaN,*Job Description Data Scientist (Full-Time)Tea...,"$75,000 - $120,000 a year",8cacfaa3c21d0129
7,8,Boston,"Data Scientist (Intern, Part-Time)",proton.ai,"Boston, MA",NaN,"Data Scientist (Intern, Part-Time)Team: Data ...",NaN,1a087273ecb3d9e3
8,9,Boston,Senior Data Scientist,SFL Scientific,"Boston, MA",NaN,SFL Scientific is expanding our Data Science C...,"$110,000 - $150,000 a year",1e7c61a0856fe452
9,10,Boston,Data Scientist,Park Jockey,"Boston, MA",NaN,Who You’ll Work For REEF Technology is the eco...,NaN,0cc7c0afb827e835


In [12]:
import math
math.isnan(test.iloc[[0]].salary)

True

In [52]:
location = test.iloc[1]['location']
location
zipcode = ''
digit_list = [s for s in location.split() if s.isdigit()]
if digit_list:
    zipcode = digit_list.pop()
# zipcode = location.endswith()
# zipcode = zc.pop()
type(zipcode)
zipcode

'02109'

In [53]:
import datetime
date = datetime.date.today()

In [54]:
date

datetime.date(2019, 9, 2)

In [55]:
str(date)

'2019-09-02'

In [101]:
URL_base = 'https://www.indeed.com/jobs?q=data+scientist+%2420%2C000'
URL_location = '&l=boston'
URL_page_start = '&start=0'
URL = URL_base + URL_location + URL_page_start

page = requests.get(URL)
time.sleep(1)  #ensuring at least 1 second between page grabs
soup = BeautifulSoup(page.text, 'lxml')
entries = get_entries(soup)

In [102]:
# soup

In [174]:
entry = entries[6]
company_info = entry.find(class_='sjcl')
location_info = company_info.find(class_='location')
location_info

# location = get_location_info(entry)
# location

<div class="location accessible-contrast-color-location">Boston, MA 02210 <span style="font-size: smaller">(South Boston area)</span></div>

In [175]:
neighborhood = location_info.find(name='span')
location = location_info.text.strip()
zipcode = ' '
if neighborhood:
    location = location.strip(neighborhood.text)
    neighborhood = neighborhood.text.strip('()')
else:
    neighborhood = ' '
temp = [ s for s in location.split() if s.isdigit() ]
if temp:
    zipcode = temp.pop()
location = location.strip(zipcode)
city_state = location.split(', ')
state = city_state.pop()
city = city_state.pop()
print(location_info.text)
# print(location_info.text.strip(neighborhood))
print(neighborhood)
print(city)
print(state)
print(zipcode)

Boston, MA 02210 (South Boston area)
South Boston area

MA 
02210


In [84]:
URL_base

'https://www.indeed.com/jobs?q=data+scientist+%2420%2C000'

# Testing Functionality

In [ ]:
URL = 'https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10'
# conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, 'html.parser')
#printing soup in a more structured tree format that makes for easier reading
# print(soup.prettify())
def extract_link_from_result(soup): 
    links = []
    entries = get_entries(soup)
    for entry in entries:
        links.append(get_link(entry))
    return(links)
def extract_summary_from_result(soup): 
    summaries = []
    entries = get_entries(soup)
    for entry in entries:
        summaries.append(get_job_summary(entry))
    return(summaries)
extract_summary_from_result(soup)

def extract_salary_from_result(soup):
    salaries = []
    entries = get_entries(soup)
    for entry in entries:
        salaries.append(get_salary(entry))
    return salaries
    
extract_salary_from_result(soup)

    
def extract_location_from_result(soup): 
    locations = []
    sub_locations = []
    rows = get_entries(soup)

    for row in rows:
        location, sub_location = get_location_info(row)
        locations.append(location)
        sub_locations.append(sub_location)
    
    return locations, sub_locations

extract_location_from_result(soup)
get_job_description('https://www.indeed.com/viewjob?jk=58bae9b6b4f17d90')
REDUNDANT:  VA Boston Healthcare System, Boston, Massachusetts
"Come join us in Boston at The VA Boston Healthcare System (VABHS), New England's premier referral center for Veterans' healthcare. VABHS is the largest recipient of VA research funds in the nation and ranked #3 among 146 VA facilities nationally for overall employee satisfaction. The Massachusetts Veterans Epidemiological Research and Information Center (MAVERIC) is an interdisciplinary research organization in VABHS. Its Informatics division is seeking a full-time Data Scientist who love working with data and want to use their data science skills to help VA researchers conduct high-impact research that expands our understanding of human health and improves healthcare outcomes for Veterans. You will join a strong, collegial, friendly, and talented team of computer scientists, statisticians, and clinicians at the VA.Core responsibilitiesUse advanced data analysis methods (from machine learning, artificial intelligence, applied statistics, etc) to assist in building predictive models of healthcare outcomes from the largest integrated medical records database in the United States, with comprehensive clinical information on over 22 million patients (2 trillion+ rows, 22,000+ columns of data), including large amounts of both structured and unstructured dataExtract, clean, and validate data in preparation for analysis, starting from raw data in a large relational databaseCarry out exploratory analyses and prepare reports summarizing our analysesCollaborate with a range of researchers, including clinicians, computer scientists and statisticians to design and execute analysesAssist in developing improved data science methodology in relation to healthcare dataAssist in publishing papers on our findings, systems, and methodologyQualificationsMUST be a US Citizen and MUST clear a US government background checkUndergraduate or graduate degree in a relevant field (computer science, statistics, informatics, physics, computational biology, etc.)Proficiency in programming with R and/or Python to carry out data analysis in large and complex datasetsExperience working with relational databases using SQL or willingness to learnAbility to move projects forward independently with moderate supportAbility to work effectively in multi-disciplinary teamsPassion for advancing healthcare research through creative use of dataStrong communication and collaborative research skills requiredExperience using version control and reproducible research (git, knitr, etc.) is preferredExperience with electronic medical record data or clinical databases preferredIf providing care for our veterans is something that appeals to you while enjoying amazing benefits, in addition to working and living in an area rich in history, culture, and recreational opportunities, this position is for you!Applicants should submit a cover letter and CV or inquiries by September 6th, 2019. Include “Data Scientist” in the e-mail subject line.Job Type: Full-timeWork Location:One locationBenefits:Health insuranceDental insuranceVision insuranceRetirement planPaid time offThis Job Is Ideal for Someone Who Is:Dependable -- more reliable than spontaneousDetail-oriented -- would rather focus on the details of work than the bigger pictureAutonomous/Independent -- enjoys working with little directi"
def extract_job_title_from_result(soup): 
    jobs = []
    # Loop over entries
    posts = get_entries(soup)
    for post in posts:
        job_title = get_job_title(post)
        jobs.append(job_title)
    return jobs
​
yobs = extract_job_title_from_result(soup)
# print(len(yobs))
yobs
/pagead/clk?mo=r&ad=-6NYlbfkN0Alc63mq7XbG_XJWtX8RBizZIwo5v3DdAY7_u3VbNSkJxVCY8YYD1XnC0LFA3U1jqXXJ3CwvsNmWRCUwfjogvYRMAC1zoKWrQruN-h5adfVpK3QtGLm_J7y0Nfc1b7UyS0PvspOMCwADQmAx0lZt2z3qf4eqYMWcLhKQjy9v-2OrynzMC6J-JiB1LU2JMnuLn0yiNMrKVuvM7JOYZqOYro4TEwS0otKLHH8DGLVfdVdXcZ7GTw4JbLpeQ-jQa7xFdo9aGta0DNafh1Jy3kVptHalREU-1fHNFr6tbUSQnwWAxt59puHK9vkmaLGEFSP1hI0WZxe6_I_6P21dOomj3-RSJDQcH7-k4giso_qVwH34lnIAVex_W8_aOPJxtFBACo7JHWV9eXyP5Q7qlKEbLpw&p=0&fvj=0&vjs=3
/pagead/clk?mo=r&ad=-6NYlbfkN0Alc63mq7XbG_XJWtX8RBizZIwo5v3DdAY7_u3VbNSkJyf6IojzDTbtcfEVZqAqVQOep1jEFzURDGO6s3JhXIcnbUPsfWI8ZQ8drUW0i30nmFJg4ufmjTupvUsdZ8n7GMzdiNYeXgj5EKqHonI0mU6qtn7TTBBLWqhaG-b5qhWBMSjCqzM8ttLTzD3MdOl8__-XLrpIt4aHJ4bWQQ1qMajSlPrIwwG0hT2hlLZEPXBYv8Xo5m-WVA1UT9CIxAXUlMdpirDc5gpZsf3c25_T1QrmhqtgYw8qgCrs8WQH0Zvy_s6UXVs-6ht_thy8YdtdwLtsmGnkthBn9btTKH5AJ_V1bELt3XM03inTev2P2MhHDuAWDlt6vhISYzzkHc1BWmp2pguJm0wUvpy4Vv5OXaQevLQEHF1e9DQ=&p=1&fvj=0&vjs=3
/pagead/clk?mo=r&ad=-6NYlbfkN0A6-B3EZN7nxI2SOKd2oJXZ-wKsUrEyzb6go4T2DhbT4oonsM92tGI0ncmlHwoyn4o_nHPRkMBDP46i_QJzmPrK_0jf8qOyZM1z3ww1hdD5e0FVigX59BZNyRfk7kr3I_FyWE9VmiyoZ65oG-04qQ9-4QIM9gGcx_5Xlyt0F2MHCyvljTyQ2pjcOTTqVF5KIyHuXFBytAlTKYGL8QBK454Q_nhkMnScpKtrVEKQF5Tw3dJmUfJePpIQRtcAxmqO0mmEGljI3Ob-XzNdSQ_5lkDjQBEljyEMiWFoXH1qPA7aj4rx6oBWuCJEojlEzYHSgzN3rtojc_SlwP03r7BK1ETQfLhmk5CSRBOtJCp0zvfLK0kVG31Lk_KZMyreBcCR-6TLEQVQwYY9gCG3DWONWdHLtmoDK97VIS4Ubmj5aw_NUAKtBJP8c6Z91e5j7VaLcVu0hx3BkrXswKdh86vgVBcm4mCSYZaE1a2ioGtxB4FH021l4AmObOHDaS_ZU4NfFVOMxrFYe0BBkfcYNAt6u7IBpIim4E7Nfh9uPxeYC1NxyXqES4a1wqylj7--yq5WdUXR_F-m0hhtuxPlV7vGJxw95xsg4WTue-A_x3m6y7Q8kOPWPnNGGHvWO1N2BiSlm4Qxbra3vHy9zZqLctqeYM2zjkZYJlKKlYzLK2-__vOOJ59lDz2HgD1wDWaDEJtxhb3jAiEhwPdYWKIYuB7BvQMUgRChoXpbM-DTjhlP_JB99r-rMkIRBpUm_k7D8S45omh63fItGj0GwURDiyiFDHXlM60Ei_cY4yNwLVPAg9_WeAPIV-FMY0Pb5ZGFAmZN7wa8M1qbOtrMW2ZjC5Efs4YjQxPvGdOH8GE9FA_Ceg61yh5ASvYyYKZDqjQS5hqpe_H18CuMZ0DMkHpHqukqBWd8MgEWtdmfYLzKKPWwFRYdrxi1UXmsAvDcuR2VbRqaUGc=&p=2&fvj=0&vjs=3
/pagead/clk?mo=r&ad=-6NYlbfkN0D19kSVUiNzG2UWy1lRGehFMusHrHGUl8ru40ax50wmt1gcnJgB2R3fXjF8kQmuiM-y1s-6cCLaWjvP1-dUQdAjDCgQGm6g-YkeMerHi4-M4xjuM1Ihhf1QZfJmPvlLE2b-Blur1k8yAKSK-JWnBpCQXirBPpUodMAre6JF1xvhRatLjVX2-WJC9s_7T1lyZd3ccHA7yjoC1UvuXa-zl-uNsaapQKROwq-zgCy4gSwi1N8qwk8KcMWzh05UoRDXpywKlKtF_SLu4yvZ0H5C_c8XjIHERJ8zaJ2eDVfo4w_7AhIzzKC9kFfUBfmc4CUqPtugN8EL8-3MEexBNM2-0UqW2zJ5efdmDkAuaUZIl7ttFh-mh-WMaukMbzwkDEI_JIvvpZnuY_r-OUGXaEGvswUlm7kTAXlzB7-l7PEy-BLn6kJFdZ8VRjYOZK6f1Uro-fufnfcZZjzZCKRJ4Gip_ovYCQFAxSmnXHglzlwthejPl-D5psfEUAxlXvKo_flXHic4D1dtxdi3jtzDYtuHRljQqX5tO3aP3vMO7QmscOSFZ1sa1jO24FOc6qaFq3kQTajlBHQyxcqY_bvoWToQvf5u6qQ22yCGi_OoCqpmxYTcZVHcAM70NDGy6_uRv9l7BRtKnTYkAzYvnJQkPA0b25oOgrZluUljlMglOyMwcnXn5vhAJ5VaAdweAu3lG0iKO1iP0Pu1DYvfPztxpTPTaJlQUmd-4EEzMX5Ekz-E928QhsBFhyP6zpcxrG8eh3GskDNLwv9HC1AAKWKN70jlj4k5IZ2lsRqEuo9QV73uoMF8R_YaqLkGez8N0IOUm_Z8GZKLQrDF7nVThQ==&p=3&fvj=0&vjs=3
/pagead/clk?mo=r&ad=-6NYlbfkN0DKhfJ5Tngw_podt8Y7TDfiYd_1ZKtIGTe6l7uxm5dGNbjU_LoFczs1I58JdWxmGrt0QHTm42DIx3dht57vtcVBXftzys82Et_agLLqB6c8Bj11U5axwXf4-7H1a120_tywtgIjmfmaUX01o7zZK3r39qCpXOzsrjjWS-d9h1CMfI3IrKDrslbgbzmyvfHHNU9LiTDF4EzrLf7pXuIv7F1Ky9f8RmEw6fsUpZd7CiWv_5UF0TsBYLNZfoVeugPgK9iYofefoPvMybZSCTeJVOrAvbvQTex_KNOtX2WYvj1Micx1yndqG11bM0GV6_6jr8K2GRrKt6Hw5BDdoGtsrIiRoCd52HVQahg9Rt4u3R0o6To7N2w-GLZMiXlYf3jxQvzfTUocy0goW_BGMb0fGbSlPmeenRO6mQGjwJgAYSxnhz9AHS67lO9Behebn30Z_nQ=&p=4&fvj=1&vjs=3
/rc/clk?jk=99129b4fc0b2ade0&fccid=4fbe585eeadd653d&vjs=3
/rc/clk?jk=6487f56b8c2629f6&fccid=b7933d099d0e4287&vjs=3
/company/Lown-Institute/jobs/Data-Scientist-ba16b29166686054?fccid=a07f39671da74cd7&vjs=3
/rc/clk?jk=fe5027e04d0757df&fccid=4ddbc06e22ec3870&vjs=3
/rc/clk?jk=297497ec8ec6b89f&fccid=5827d5f055ef093a&vjs=3
/rc/clk?jk=06815d4a376f8547&fccid=b3fc015a7c99e39b&vjs=3
/rc/clk?jk=5a64eea9c1f2a2d2&fccid=3fc1e7f5af93d293&vjs=3
/rc/clk?jk=e2646d60162b5288&fccid=deb234f9dd3edcea&vjs=3
/pagead/clk?mo=r&ad=-6NYlbfkN0AHdFIJfBLGYbx_TzBCBsgbK0Due77H36xmGRpY2MfZsJoW53LfYfjCE-IzkCRwATj29afdE3nXUxDkNOWpb9F8DqmXvlw6JI4zwd-Qv44Jg-VAgVr_QRXZYTSFmIZOrFpPdsJkCQmOtaR11KMr7trdCOGtpPvqHGBZjpOpltn2FmMcR8fSmQ3gfVkJXncXuLSuE0kSbOyKNTywG1tzsCh7G6cp5fTAkwdLCJO8WyQQWQ0vJeZBsJIAj0dkV_IOVbgHr3dQAkXJP-NRY-rv-oEiXmrqDu-JPbmgunUVRNwsJhStyASs_L_H7nJQyaHkSHUUcGNe8FNCwzkQP10pm0E6mrBXoaTEDQfcQyET5YaeWXI2uB6-HWDSb7wZRMxmRz9f9WtmCQDdsiI37c4C-TxgyLfwBXirKRn9EIAAf20JqH9XwY7AUyfrTGcEI849AdbOYj-L4ebCRm8xAud7BJq_BWWqXYD-WJ-yt_LlRuYiRQ==&p=13&fvj=1&vjs=3
/pagead/clk?mo=r&ad=-6NYlbfkN0AkmEfxfIvywYkr7FPgbbzZtS80TzOGC0Qwk8jq8kfwxcmRMEv4cXPur2iIiy6S-b2A9S63iQ6y273o04DbbDa8DGeXjpHpilpYvUecdh3-r0EnPObskLpHWJaFb7yLqEv7kpqw5OGXyhjkdQmikH0IjiKjrZQlqkEg1HladOOsBwwKb6bm_vhRtM6SptFSdDJBS_mJxTdtr01YKk1vxh4mHGjuCjwtaQUmceQ8kbVS6uwKVCAmkqjeHsZ6zECtj-_y9pA6jrtX2dJRrcfEiTgSxl74HlWfm8frNYb0tLqoDAVLHTYvvABDwipAfAIJ2LSoCiXDhVDaNJyHYNJElDa8lJoWZFfsvu6j7tKkJXO-zC9ZzD9frxo-Jp562ubvbR6IX3VREj8uVeEqmYUudkUlRT15xMYqEhoWHiyNpeaB5u3IHgl1cSVRVJhKLxuI0lM=&p=14&fvj=1&vjs=3
/pagead/clk?mo=r&ad=-6NYlbfkN0Al2NfJ8yyqIYYnTh93X-NBwjrShfZHouU-86CQVQRn0mOcjHfraeGWRogexiUmwZruqiI824jnWs5HwzuMsSuDttoJig44REKU1oxidI8Bib-8bUI6AAVeV1BKl8zs8Lqjs2tvJ9Clj_-2PV3YSEhyt3kuCERpjC9DaR7o5a9uBeFwAlucLb7w6y7YbvrIbuG-2d7-jkkfbu9o1XOTkusLEy53mYYIKUnb5F_RKc5it2RkB6Y-yTJQ9qLByW3RnOZwYoI7l2_QIte_Mrlc_qdmBpMUlvtAxLXGq_iqjm_exojP1IoKlTL9rpZFr6SFbtFy1bP_IBnF0ZahXnPJQVWXU5ryzQUjr5FpBIXg1n1XPQM3b8HtjtZiXr0438Yf0P5_c5vnqIIB3KxQPYPcjaXcp9d5n4f9Zul9LPKDjWGn5hGVc8AQDcLR&p=15&fvj=1&vjs=3
['Translational Research Scientist (with proteomic mass spec e...',
 'Scientist, Neurodegenerative Diseases',
 'Data Scientist',
 'Assistant Director, Data Science',
 'Data Scientist',
 'Data Scientist',
 'Linked Data Consultant / Field Application Scientist - Junio...',
 'Data Scientist',
 'Data Scientist',
 'LSP Board Scientist/Investigator',
 'Data Scientist',
 'Data Scientist (Labs)',
 'Data Scientist, NLP',
 'Perception Scientist for Marine Autonomy',
 'Data Analyst II',
 'Data Scientist']
def extract_company_from_result(soup): 
    companies = [] 
    for entry in soup.find_all(name='div', attrs={'class':'row'}): 
        company = get_company(entry)
        companies.append(company) 
    return companies
​
extract_company_from_result(soup)
Possible U

In [ ]:

def get_job_description(job_page):
    page = requests.get(job_page)
    time.sleep(1)  #ensuring at least 1 second between page grabs
    soup = BeautifulSoup(page.text, 'lxml')

    # Loop over posts/entries
    description = soup.find(name='div', id='jobDescriptionText')
    
    # This would be useful if all of the postings were the same format
    # The idea is to get rid of the redundant information like company name and location
#     redundant_info = description.find(name='p').text
#     print('REDUNDANT: ', redundant_info)
#     description = description.find_all(name='p')
#     description = description[1:]
#     description = [item for sublist in description[1:] for item in sublist]
    description = description.text.strip()
    description = description.replace('\n',' ')
    description = description.replace('\t',' ')
    return description
